Create IGC
=======

This creates a ISS orbit, and a pushbroom camera. The camera is really different than the real camera (which is a push whisk camera). But as first order this should be vaguely realistic.

In [1]:
from geocal import *

Get the ASTER mosaic we are working with, and subset to the area of interest (This came from Tom and is just the northern part of CA).

We have b4 is approx the SWIR 1.6 band of ecostress, b6 is SWIR 2.2.

In [2]:
f = VicarLiteRasterImage("/raid27/tllogan/ecostress/coregtst/calnorm_b4.img")
f = SubRasterImage(f,0,0,37000, 40000)

In [3]:
print f.ground_coordinate(ImageCoordinate(f.number_line / 2, f.number_sample / 2))

Geodetic: (39.4444 deg, -122.236 deg, 0 m)


We inspected http://heavens-above.com looking for tracks that look like it goes near this point. It looks like oneof the passes from 10/23/2015 (PST) looks pretty good for this. Note also data 10/3/2015 which goes across the other way, if we need to use that instead.

In [4]:
# Time here is the maximum altitude for our point. It is given in local time, so we need to
# convert to UTC
t = Time.parse_time('2015-10-23T19:02:38Z') + 7 * 60 * 60
print t

2015-10-24T02:02:38.000000Z


In [5]:
# TLE comes from http://www.n2yo.com/satellite/?s=25544. This was for 10/24/2015
tle = '''\
1 25544U 98067A   15301.64187079  .00010243  00000-0  15860-3 0  9995
2 25544  51.6438 138.0120 0006887  88.7083 356.4634 15.54663652968802
'''
orb = TleOrbit(tle)

In [6]:
print Geodetic(orb.position_cf(t))

Geodetic: (39.4465 deg, -122.196 deg, 403997 m)


Focal length and ccd pixel size comes from Eugene's SDS data bible. The scaling of the CCD size is empirical to give the right resolution on the surface. These are pretty hoaky, we really just want something vaguely right since our camera model is pretty different from a pushbroom. But this gives a place to start.

In [7]:
frame_to_sc = quat_rot("ZYX", 0, 0, 0)
cam = QuaternionCamera(frame_to_sc, 1, 5400, 40e-3 * 1.8, 40e-3 * 2, 425, FrameCoordinate(1.0 / 2, 5400.0 / 2))
print cam

QuaternionCamera:
   Number line:     1
   Number sample:   5400
   Focal length:    425 mm
   Line pitch:      0.072 mm
   Sample pitch:    0.08 mm
   Principal point: Frame Coordinate: (0.5, 2700)
   Frame convention: LINE_IS_X
   Frame to spacecraft: (1,0,0,0)



The time table data comes from Eugene's SDS data bible file (ECOSTRESS_SDS_Data_Bible.xls in ecostress-sds git repository). The real camera is a bit complicated, but we collect about 241 samples of data (in along track direction) every 1.181 seconds. For a pushbroom, we can divide this up evenly as an approximation. But then there is an averaging step (which I don't know the details of) that combines 2 pixels. So we have the factor of 2 given. Scene has 5400 pixels, which is where time calcuation comes from.

In [8]:
# False here says it ok for SrtmDem to not have tile. This gives support
# for data that is over the ocean.
dem = SrtmDem("",False)
band = 0
tspace = 1.181 / 241 * 2
toff = 5400 * tspace / 2
tt = ConstantSpacingTimeTable(t - toff + tspace, t + toff, tspace)
ipi = Ipi(orb, cam, band, tt.min_time, tt.max_time, tt)
igc = IpiImageGroundConnection(ipi, dem, None)

In [9]:
print distance(igc.ground_coordinate(ImageCoordinate(1, 5400 / 2)), 
               igc.ground_coordinate(ImageCoordinate(1, 1 + 5400 / 2)))
print distance(igc.ground_coordinate(ImageCoordinate(1, 5400 / 2)), 
               igc.ground_coordinate(ImageCoordinate(2, 5400 / 2)))

78.2246189551
67.963124052


In [10]:
print igc.footprint_resolution(1, 5400 / 2)

(68.45938212612417, 76.06608503100865)


In [11]:
write_shelve("igc_test.xml", igc)
write_shelve("aster_rad.xml", f)
# We may want to have IgcSimulated not read everything into memory. But start with this
write_shelve("test_data.db:igc_simulated", IgcSimulated(igc, f, -1, False))
# Then do write_image --verbose --number-process=20 --process-number-line=100 --process-number-sample=100 test_data.db:igc_simulated l1b1_sim.img 

In [12]:
print f.coordinate(igc.ground_coordinate(ImageCoordinate(0,0)))
print f.coordinate(igc.ground_coordinate(ImageCoordinate(0,igc.number_sample - 1)))
print f.coordinate(igc.ground_coordinate(ImageCoordinate(igc.number_line - 1, 0)))
print f.coordinate(igc.ground_coordinate(ImageCoordinate(igc.number_line - 1, igc.number_sample - 1)))

(249.462, 18524.1)
(22085, -1894.57)
(15371.8, 42765.2)
(36555.8, 21889.5)


In [13]:
write_shelve("orbit.xml", orb)
write_shelve("camera.xml", cam)
write_shelve("time_table.xml", tt)